In [1]:
import pickle
import onnxruntime as rt

with open("input_t.pkl", "rb") as f:
    input_t = pickle.load(f)

original_model_path = "original_model.onnx"
session = rt.InferenceSession(model_path, providers=['CPUExecutionProvider'])

In [ ]:
import onnxruntime as rt
import numpy as np

def create_and_run_onnx_session(model_path: str, input_data: np.ndarray):
    """
    Creates an ONNX Runtime session and runs inference on a given ONNX model.

    Args:
        model_path (str): The file path to the ONNX model.
        input_data (np.ndarray): The input data for the model,
                                 formatted as a NumPy array.
                                 Ensure its shape and dtype match the model's expected input.

    Returns:
        list: The output(s) from the ONNX model inference.
              Returns None if an error occurs.
    """
    try:
        # Create an inference session
        # You can specify providers, e.g., providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
        # to prioritize GPU if available.
        session = rt.InferenceSession(model_path, providers=['CPUExecutionProvider'])
        print(f"ONNX session created successfully for model: {model_path}")

        # Get input and output names
        input_name = session.get_inputs()[0].name
        output_name = session.get_outputs()[0].name
        print(f"Model input name: {input_name}")
        print(f"Model output name: {output_name}")

        # Run inference
        # The run method takes a list of output names (or None for all outputs)
        # and a dictionary of input names to input data.
        outputs = session.run([output_name], {input_name: input_data})
        print("Inference completed successfully.")

        return outputs

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

if __name__ == "__main__":
    # --- Example Usage ---
    # 1. Create a dummy ONNX model for demonstration purposes.
    #    In a real scenario, you would load an existing .onnx file.
    #    This part requires onnx and numpy to be installed.

    # This is a very simple dummy model: input -> add 1.0 -> output
    try:
        import onnx
        from onnx import helper
        from onnx import TensorProto

        # Define the graph (nodes, inputs, outputs)
        # Input: 'input_tensor' (float32, shape [1, 3, 224, 224])
        input_tensor = helper.make_tensor_value_info('input_tensor', TensorProto.FLOAT, [1, 3, 224, 224])
        # Output: 'output_tensor' (float32, same shape as input)
        output_tensor = helper.make_tensor_value_info('output_tensor', TensorProto.FLOAT, [1, 3, 224, 224])

        # Node: Add constant 1.0 to input_tensor
        one_const = helper.make_tensor('one_const', TensorProto.FLOAT, [], [1.0]) # Scalar 1.0
        node_def = helper.make_node(
            'Add',
            ['input_tensor', 'one_const'], # Inputs to the Add operation
            ['output_tensor'],             # Output of the Add operation
            name='add_one_node'
        )

        # Graph: Define the model graph
        graph_def = helper.make_graph(
            [node_def],
            'simple_add_model',
            [input_tensor],
            [output_tensor],
            [one_const] # Initializers (constants) used in the graph
        )

        # Model: Define the ONNX model
        model_def = helper.make_model(graph_def, producer_name='onnx-example')
        model_def.opset_import[0].version = 13 # Set opset version

        onnx_model_path = "dummy_model.onnx"
        onnx.save(model_def, onnx_model_path)
        print(f"\nDummy ONNX model saved to {onnx_model_path}")

        # 2. Prepare dummy input data
        dummy_input = np.random.rand(1, 3, 224, 224).astype(np.float32)
        print(f"Dummy input data shape: {dummy_input.shape}, dtype: {dummy_input.dtype}")

        # 3. Run the function with the dummy model and input
        print("\n--- Running ONNX session ---")
        model_outputs = create_and_run_onnx_session(onnx_model_path, dummy_input)

        if model_outputs is not None:
            print("\nModel output(s):")
            for i, output in enumerate(model_outputs):
                print(f"Output {i} shape: {output.shape}, dtype: {output.dtype}")
                # print(f"Output {i} (first 5 elements): {output.flatten()[:5]}")
                # You can compare with expected output: dummy_input + 1.0
                # print(f"Expected output (first 5 elements): {(dummy_input + 1.0).flatten()[:5]}")

    except ImportError:
        print("\n'onnx' library not found. Skipping dummy model creation.")
        print("Please install it (`pip install onnx`) if you want to run the dummy example.")
        print("You can still use the `create_and_run_onnx_session` function with your own .onnx file.")
    except Exception as e:
        print(f"\nAn error occurred during dummy model creation or execution: {e}")

    # To run with your own model:
    # my_model_path = "path/to/your/model.onnx"
    # my_input_data = np.random.rand(1, 10).astype(np.float32) # Adjust shape and dtype for your model
    # my_outputs = create_and_run_onnx_session(my_model_path, my_input_data)
    # if my_outputs:
    #     print("Your model outputs:", my_outputs)
